In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random
import time
import math

In [ ]:
from nd_code import *

# Simple Testing

In [ ]:
G = nx.erdos_renyi_graph(100, 0.1)
# G = susc_generator(20, 2, p=0)
# G = nx.random_lobster(200, 0.15, 0.15)
for n in G.nodes:
  G.nodes[n]['node weight'] = 100 * (1 - random.randrange(0, 1))

for e in G.edges:
  u, v = e
  G.edges[e]['edge weight'] = G.nodes[u]['node weight'] + G.nodes[v]['node weight']

In [ ]:
C = 100
node_size=50

## Compare GND against AB

S = gnd(G, C, verbose=False)
print(f"Number of nodes removed: {len(S)}")
print(f"Nodes removed: {S}")
plt.title(f"GND Starting Graph")
draw_graph_with_nodes(G, S, node_size=node_size)
plt.figure()
plt.title(f"GND Dismantled Graph")
g_c2 = G.copy()
g_c2.remove_nodes_from(S)
draw_graph_ccs(g_c2,node_size=node_size)
plt.figure()

print(f"------------")

S_ab, _ = greedy_algorithm(approximate_betweenness_centrality_cd, G, c=C / G.number_of_nodes())
print(f"Nodes removed: {S_ab}")
plt.title(f"AB Starting Graph")
draw_graph_with_nodes(G, S_ab,node_size=node_size)
plt.figure()
plt.title(f"AB Dismantled Graph")
g_c2 = G.copy()
g_c2.remove_nodes_from(S_ab)
draw_graph_ccs(g_c2,node_size=node_size)